# RISC-V Qemu

Ya que están agotados los CPUs RISCV64 disponibles, debemos usar Qemu con los últimos partes de RISC-V para correr nuestras distribuciones.

Siguiendo esta Libreta puedes tener un ambiente de desarrollo y pruebas listo y funcionando en muy poco tiempo.

Nota: Esta guía es un trabajo en desarrollo activo.

Este tutorial está basado en [Riscv Qemu en Github](https://github.com/riscv/riscv-qemu) y el blog sobre [el stage4 de Fedora](https://rwmj.wordpress.com/2018/02/15/fedora-risc-v-runnable-stage-4-disk-images/).

Mientras espero por un CPU RISCV64 para jugar con opciones de compilación y desempeño, estoy utilizando servidores poderosos patrocinados por [Greencore Solutions](https://www.greencore.co.cr/) y que corren Ubuntu. Si necesita ayuda haciendo funcionar esta guía en otras distribuciones, favor hacérmelo saber para agregarle información a este tutorial.

Comenzamos por instalar paquetes básicos.

In [ ]:
cd ~
sudo apt-get install -y git flex bison lftp
sudo apt-get build-dep -y qemu

Ahora descargamos el código, creamos un directorio para compilado, y ejecutamos configure, make y make install.

In [ ]:
git clone https://github.com/riscv/riscv-qemu.git
cd riscv-qemu
mkdir build
cd build
../configure
make -j6
sudo make install

Descargamos el stage4 de Fedora. Este paso es necesario mientras se completa el stage4 de Slacware RISCV64.

In [ ]:
cd ~
mkdir fedora-stage4
cd fedora-stage4
lftp -e "mirror -c;exit" https://fedorapeople.org/groups/risc-v/disk-images/

Descomprimimos el stage4 para hacerlo disponible a Qemu, y creamos un segundo disco para desarrollo.

In [ ]:
xz -dk stage4-disk.img.xz
fallocate -l 40G slack.img
mkfs.ext4 slack.img

**Desde una terminal**:
```
sudo qemu-system-riscv64 \
  -nographic -machine virt -m 6G -smp 6 \
  -kernel bbl \
  -append "console=ttyS0 ro root=/dev/vda init=/usr/sbin/init" \
  -device virtio-blk-device,drive=hd0 \
  -drive file=stage4-disk.img,format=raw,id=hd0 \
  -device virtio-net-device,netdev=usernet \
  -netdev tap,id=usernet \
  -device virtio-blk-device,drive=hd1 \
  -drive file=slack.img,format=raw,id=hd1
```

Luego de que el stage4 termine de arrancar, ya se puede conectar vía ssh.

Yo recomendaría agregar estas herramientas básicas a Fedora
```
dnf install -y vim screen git bison byacc rsync libtool ncurses-devel gettext attr python2 ccache \
  libuuid-devel expat-devel libcap-ng-devel zlib-devel device-mapper-devel popt-devel libblkid-devel \
  libgcrypt-devel python2-devel flex bzip2-devel lzip lzo-devel gmp-devel libmpc-devel mpfr-devel \
  python3-devel curl curl-devel libarchive-devel qt5-devel automake help2man gtk2-devel cmake \
  openssl-devel asciidoc xmlto binutils-devel xapian-core xapian-core-devel sqlite \
  sqlite-devel qt5-devel sqlite-devel qt5-devel qt5-qtmultimedia-devel sqlite-devel perl-XML-Parser \
  texinfo gcc-c++ wget libffi-devel mpdecimal-devel libtool-ltdl-devel libunistring-devel \
  readline-devel libdwarf-devel guile-devel gnupg2 gnupg libksba-devel libassuan-devel \
  npth-devel gdbm-devel libpcap libpcap-devel libmnl-devel libnftnl-devel \
  libnetfilter_conntrack-devel libnetfilter_conntrack

```

Para desarrollo de Slackware, ejecutamos dentro del stage4 de Fedora:
```
cat << EOF >> /etc/fstab
/dev/vdb /mnt ext4 defaults 0 0
tmpfs   /tmp         tmpfs   nodev,nosuid,size=9G          0  0
EOF
mount -a
cd /mnt/
git clone https://github.com/fede2cr/slackware_riscv.git
cd /
tar xJvf /mnt/slackware_riscv/slackware-current/slackware/a/pkgtools-15.0-noarch-6.txz
export PATH="/usr/lib64/ccache/:$PATH:/sbin"
export NUMJOBS=" -j6 "
mkdir /mnt/chroot
/mnt/slackware_riscv/people/fede2/tools/build_chroot.sh
```